In [46]:
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import itertools
import pulp as pl
import time
import ast
import os

In [47]:
class BipartiteGraph:
    
    def __init__(self, nU, nW, edges, name):
        self.name = name # gráf neve, pl. K_3,3
        self.nU = nU # egyik csúcsosztály mérete
        self.U = [i for i in range(0,nU)]
        self.nW = nW # másik csúcsosztály mérete
        self.W = [i for i in range(nU,nU+nW)]
        self.edges = [(e[0],e[1]+nU) for e in edges] # élek listája
        j=0
        self.edge_dict={} # él -> id
        self.inc = {} # csúcs -> rá ill. élek listája
        for i in range(0,nU+nW):
            self.inc[i]=[]
        for edge in self.edges:
            self.inc[edge[0]].append(edge[1])
            self.inc[edge[1]].append(edge[0])
            self.edge_dict[edge] = j
            j += 1

    def make_vectors(self, edge_subset): # élhalmazhoz tartozó vektorhalmaz kiszámítása
        vectors=[]
        for e in edge_subset:
            vector=[0 for i in range(0,self.nU+self.nW)]
            vector[e[0]]=1
            vector[e[1]]=-1
            vectors.append(vector)
        return(vectors)

    def all_spanning_trees(self): # összes feszítőfa megkeresése
        print("  keressuk a feszitofakat...")
        spanning_trees = {} # tuple a ffa éleivel -> fához tartozó vektor
        degree_sequences_U = {} # fokszám sorozat U-n -> ezt realizáló feszítőfák száma
        degree_sequences_W = {} # csak ellenőrzésre: fokszám sorozat W-n -> ezt realizáló feszítőfák száma
        tree_to_sequence = {} # feszítőfa -> hozzá tartozó fokszámsorozat
        for edge_subset in itertools.combinations(self.edges, self.nU+self.nW-1): # minden (n-1)-es élrészhalmazra megnézzük bfs-sel, hogy összefüggő-e
            visited=set()
            stack=[edge_subset[0][0]]
            while stack:
                vertex=stack.pop()
                #print("vizsgalt csucs:",vertex,"\nszomszedok:")
                if vertex not in visited:
                    visited.add(vertex)
                    for v in self.inc[vertex]:
                        #print(v)
                        if ((vertex,v) in edge_subset or (v,vertex) in edge_subset) and v not in visited:
                            stack.append(v)
            if len(visited)==self.nU+self.nW: # ha összefüggő
                spanning_trees[edge_subset]=self.make_vectors(edge_subset)
                degree_sequence_U = [0 for i in range(self.nU)]
                degree_sequence_W = [0 for i in range(self.nW)]
                for edge in edge_subset:
                    degree_sequence_U[edge[0]] += 1
                    degree_sequence_W[edge[1]-self.nU] += 1
                tree_to_sequence[edge_subset] = tuple(degree_sequence_U)
                if tuple(degree_sequence_U) in degree_sequences_U:
                    degree_sequences_U[tuple(degree_sequence_U)] += 1
                else:
                    degree_sequences_U[tuple(degree_sequence_U)] = 1
                if tuple(degree_sequence_W) in degree_sequences_W:
                    degree_sequences_W[tuple(degree_sequence_W)] += 1
                else:
                    degree_sequences_W[tuple(degree_sequence_W)] = 1
        if len(degree_sequences_U)!=len(degree_sequences_W): # ellenőrzés: fokszámsorozatok száma megegyezik
            print("  valami baj van (fokszámsorozatok száma nem egyezik)")
        print("     megvannak a feszitofak", round(time.time()-start_time,2))
        print("  feszitofak szama:",len(spanning_trees))
        return(spanning_trees,len(degree_sequences_U),tree_to_sequence)

    def two_trees_disjoint(self,tree1_vectors,tree2_vectors): # két fához tartozó politópoknak van-e közös belső pontja
        prob = pl.LpProblem(name="disjoint_trees",
                            sense=pl.LpMinimize)
        idx=[i for i in range(self.nU + self.nW - 1)]
        x = pl.LpVariable.dicts(name="x",
                                lowBound=0.0001, # epszilon
                                indices=idx,
                                cat=pl.LpContinuous)
        y = pl.LpVariable.dicts(name="y",
                                lowBound=0.0001,  # epszilon
                                indices=idx,
                                cat=pl.LpContinuous)
        for i in range(self.nU+self.nW):
            prob += pl.lpSum(x[j]*tree1_vectors[j][i]-y[j]*tree2_vectors[j][i] for j in idx) == 0
        prob += pl.lpSum(x) == 1
        prob += pl.lpSum(y) == 1
        prob.solve(pl.PULP_CBC_CMD(msg=False))
        return(prob.status) # 0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'

    def k_trees_disjoint(self,tree_set,tree_to_sequence): # nem használom
        # igaz-e, hogy a tree_setben kulonbozoek a fokszamsorozatok
        degrees=set()
        for tree in tree_set:
            degrees.add(tree_to_sequence[tree])
        if len(degrees) == len(tree_set): # minden fokszámsorozat különböző
            return 1 # még lehet háromszögelő
        else:
            return 0 # biztosan nem háromszögelő
        
    def find_triangulations(self): # összes háromszögelő fa halmaz megkeresése
        spanning_trees,nSimplex,tree_to_sequence = self.all_spanning_trees()
        print("  szimplexek szama:",nSimplex)
        print("  vizsgalunk minden fa part...")
        intersecting_tree_graph={} # feszítőfa -> vele összemetsző feszítőfák halmaza
        for tree in spanning_trees.keys():
            intersecting_tree_graph[tree]=set()
        for two_trees in itertools.combinations(spanning_trees.keys(), 2): # minden fa párra megnézzük, hogy diszjunktak-e
            if tree_to_sequence[two_trees[0]] == tree_to_sequence[two_trees[1]]: # ha uaz a fokszámsorozatuk --> biztosan metszőek
                intersecting_tree_graph[two_trees[0]].add(two_trees[1])
                intersecting_tree_graph[two_trees[1]].add(two_trees[0])
            elif self.two_trees_disjoint(spanning_trees[two_trees[0]],spanning_trees[two_trees[1]]) == 1: # ha kül a fokszámsorozatuk --> lp-vel eldöntjük, hogy metszőek-e 
                intersecting_tree_graph[two_trees[0]].add(two_trees[1])
                intersecting_tree_graph[two_trees[1]].add(two_trees[0])
        print("     megvan az osszefuggosegi graf", round(time.time()-start_time,2))
        triang={} # háromszögelés -> vektor, i-edik koordinátája: i-edik él hányszor szerepel a háromszögelésben
        print("  vizsgalunk minden potencialis haromszogelest...")
        for tree_subset in itertools.combinations(spanning_trees.keys(), nSimplex): # minden megfelelő méretű részhalmazra megnézzük, hogy háromszögelő-e
            check_subset=1
            #if self.k_trees_disjoint(tree_subset,tree_to_sequence) == 0: # ez csak lassította a programot
                #check_subset=0 # biztosan nem háromszögelő
            #else:
            for two_trees in itertools.combinations(tree_subset, 2):
                if two_trees[1] in intersecting_tree_graph[two_trees[0]]: # bármely két ffa politóp metsző --> nem háromszögelő
                    check_subset = 0
                    break
            if check_subset==1:
                vector = [0 for e in self.edges]
                for tree in tree_subset:
                    for edge in tree:
                        vector[self.edge_dict[edge]] += 1
                triang[tree_subset] = vector
        print("     megvannak a haromszogelesek", round(time.time()-start_time,2))
        return(triang,len(spanning_trees))
    
    def vertices(self,triang): # kiválogatja a poliéder csúcsait
        vertices=set()
        for t in triang.keys():
            prob = pl.LpProblem(name="is_vertex")
            idx=[]
            for k in triang.keys():
                if k!=t:
                    idx.append(k)
            x = pl.LpVariable.dicts(name="x",
                                    lowBound=0,  # epszilon
                                    indices=idx,
                                    cat=pl.LpContinuous)
            for i in range(len(self.edges)):
                prob += pl.lpSum(x[k] * triang[k][i] for k in idx) == triang[t][i]
            prob += pl.lpSum(x) == 1
            prob.solve(pl.PULP_CBC_CMD(msg=False))
            if prob.status==-1: # 0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'
                vertices.add(t)
        return (vertices)

    def draw_triangulations(self,triang,ntrees,draw,distr,reg,idx,filename): # kirajzoljuk a reguláris és nem reguláris háromszögelő halmazokat, és a fákat előfordulási gyakoriságuk szerint
        all = len(triang.keys())
        if reg==1: # reguláris háromszögelések kiválogatása, és ha draw==1, akkor pirosra színezése
            print("  csucs haromszogelesek kivalogatasa...")
            vertices = self.vertices(triang)
            print("     megvannak a csucs haromszogelesek", round(time.time()-start_time,2))
        if distr == 1: # fák gyakoriságának meghatározása, és kirajzolása
            if not os.path.exists(os.path.join(os.getcwd(), 'distribution'+'_'+filename+'_'+str(idx))):
                os.mkdir(os.path.join(os.getcwd(), 'distribution'+'_'+filename+'_'+str(idx)))
            count_trees = {}
        if draw == 1: # háromszögelő halmazok kirajzolása kékkel
            if not os.path.exists(os.path.join(os.getcwd(), 'triangulations'+'_'+filename+'_'+str(idx))):
                os.mkdir(os.path.join(os.getcwd(), 'triangulations'+'_'+filename+'_'+str(idx)))
            print("  keszulnek az abrak...")
            nplot = 0
        for t in triang.keys():
            if draw==1:
                nfig = 0
                fig, axs = plt.subplots(len(t), figsize=(15,15))
                fig.suptitle(str(idx+1)+'. gráf ('+self.name+')'+'\nháromszögelés: '+ str(nplot+1)+'/'+str(all), fontsize=20)
            for edge_subset in t:
                if draw == 1:
                    if len(t)>1:
                        axs[nfig].axis('off')
                        for i in range(self.nU):
                            axs[nfig].scatter(i,1,color='blue',s=100)
                        for i in range(self.nW):
                            axs[nfig].scatter(i,0,color='blue',s=100)
                        for edge in self.edges:
                            #print(edge)
                            axs[nfig].plot([edge[0],edge[1]-self.nU], [1,0], color='lightgrey')
                        for edge in edge_subset:
                            #print(edge)
                            if reg==1 and t in vertices:
                                axs[nfig].plot([edge[0],edge[1]-self.nU], [1,0], color='red',linewidth=3)
                            else:
                                axs[nfig].plot([edge[0], edge[1] - self.nU], [1, 0], color='blue', linewidth=3)
                        nfig+=1
                    else: # csak azért kell, hogy fákra is működjön
                        axs.axis('off')
                        for i in range(self.nU):
                            axs.scatter(i,1,color='blue',s=100)
                        for i in range(self.nW):
                            axs.scatter(i,0,color='blue',s=100)
                        for edge in self.edges:
                            #print(edge)
                            axs.plot([edge[0],edge[1]-self.nU], [1,0], color='lightgrey')
                        for edge in edge_subset:
                            #print(edge)
                            if reg==1 and t in vertices:
                                axs.plot([edge[0],edge[1]-self.nU], [1,0], color='red',linewidth=3)
                            else:
                                axs.plot([edge[0], edge[1] - self.nU], [1, 0], color='blue', linewidth=3)
                if distr==1:
                    if edge_subset in count_trees.keys():
                        count_trees[edge_subset] += 1
                    else:
                        count_trees[edge_subset] = 1
            if draw==1:
                fig.savefig('triangulations'+'_'+filename+'_'+str(idx)+'/plot' + str(idx) + '_' + str(nplot) + '.png')
                plt.close(fig)
                nplot+=1
        #if draw==1:
            #plt.show()
        if distr==1:
            nplot=0
            sorted_trees=sorted(count_trees.items(), key=lambda x: x[1], reverse=True)
            for tree,count in sorted_trees:
                fig, axs = plt.subplots(1,1, figsize=(15, 15))
                fig.suptitle('előfordulás: ' + str(count)+'/' + str(all) + ' (' + "{:.2f}".format((count/all)*100)+'%)\n'+str(idx+1)+'. gráf: ('+self.name+')', fontsize=20)
                axs.axis('off')
                for i in range(self.nU):
                    axs.scatter(i, 1, color='blue', s=100)
                for i in range(self.nW):
                    axs.scatter(i, 0, color='blue', s=100)
                for edge in self.edges:
                    # print(edge)
                    axs.plot([edge[0], edge[1] - self.nU], [1, 0], color='lightgrey')
                for edge in tree:
                    # print(edge)
                    axs.plot([edge[0], edge[1] - self.nU], [1, 0], color='blue', linewidth=3)
                nplot+=1
                fig.savefig('distribution'+'_'+filename+'_'+str(idx)+'/plot'+ str(idx) + '_' + str(nplot) + '.png')
                plt.close(fig)
                print(' ',tree,"--",count)
            if len(count_trees)!=ntrees:
                print(ntrees-len(count_trees)," darab fa nincs haromszogelesben")
            print("     abrak kesz",round(time.time()-start_time,2))


In [48]:
def main(filename,draw,distr,reg):
    bipartite_graphs = []
    with open(filename, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 3):
            name = lines[i].strip()
            n, m = map(int, lines[i+1].split())
            edges = ast.literal_eval(lines[i+2])
            bipartite_graph = BipartiteGraph(n, m, edges, name)
            bipartite_graphs.append(bipartite_graph)
    idx=0
    for G in bipartite_graphs:
        print(G.name+' ('+str(idx)+')')
        print('  elek: ',G.edges)
        print(' ',round(time.time()-start_time))
        triang,ntrees = G.find_triangulations()
        G.draw_triangulations(triang, ntrees, draw, distr, reg, idx, filename)
        idx+=1

In [72]:
# Itt kell futtatni a programot

start_time=time.time()
main("pelda.txt",1,1,1)
#main("K_2_2.txt",1,1,1)
#main("osszefuggo_nemfa_paros_grafok_3_3.txt",1,1,1)
#main("osszefuggo_nemteljes_nemfa_paros_grafok_3_3.txt",1,1,1)
#main("osszefuggo_nemfa_paros_grafok_4_2.txt",1,1,1)
#main("osszefuggo_nemfa_paros_grafok_5_2.txt",1,1,1)
#main("K_4_2-e_cseresznyevel.txt",1,1,1)
print("vege", round(time.time() - start_time, 2))

K_3,3 - e (0)
  elek:  [(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4)]
  0
  keressuk a feszitofakat...
     megvannak a feszitofak 0.01
  feszitofak szama: 36
  szimplexek szama: 5
  vizsgalunk minden fa part...
     megvan az osszefuggosegi graf 10.56
  vizsgalunk minden potencialis haromszogelest...
     megvannak a haromszogelesek 10.75
  csucs haromszogelesek kivalogatasa...
     megvannak a csucs haromszogelesek 11.76
  keszulnek az abrak...
  ((0, 5), (1, 4), (1, 5), (2, 3), (2, 4)) -- 11
  ((0, 5), (1, 3), (1, 5), (2, 3), (2, 4)) -- 11
  ((0, 4), (0, 5), (1, 5), (2, 3), (2, 4)) -- 11
  ((0, 3), (0, 5), (1, 5), (2, 3), (2, 4)) -- 11
  ((0, 3), (0, 4), (0, 5), (1, 3), (2, 3)) -- 9
  ((0, 4), (1, 3), (1, 4), (1, 5), (2, 4)) -- 9
  ((0, 3), (1, 3), (1, 4), (1, 5), (2, 3)) -- 9
  ((0, 3), (0, 4), (0, 5), (1, 4), (2, 4)) -- 9
  ((0, 4), (0, 5), (1, 4), (2, 3), (2, 4)) -- 8
  ((0, 5), (1, 3), (1, 4), (1, 5), (2, 3)) -- 8
  ((0, 5), (1, 3), (1, 4), (1, 5), (2, 4)) -- 8